In [16]:
#pip install spotipy pandas Dont forget!

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

# Your Spotify API credentials 👇
client_id = '' #YOUR CLIENT ID
client_secret = '' #YOUR SECRET ID

# Authenticate
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

# Extract artist ID from URL
def extract_artist_id(url):
    if 'artist/' in url:
        return url.split('artist/')[1].split('?')[0]
    return None

# Example artist URL
artist_url = ''  # PASTE YOUR ARTIST URL HERE
artist_id = extract_artist_id(artist_url)

# Fetch artist info
artist_info = sp.artist(artist_id)
artist_name = artist_info['name']
artist_followers = artist_info['followers']['total']

# Get all albums (including singles + compilations)
albums = sp.artist_albums(artist_id, album_type='album,single,compilation', country='US', limit=50)
album_ids = list(set([album['id'] for album in albums['items']]))

# Get all tracks from all albums
all_tracks = []
for album_id in album_ids:
    tracks = sp.album_tracks(album_id)
    for track in tracks['items']:
        track_info = sp.track(track['id'])
        all_tracks.append({
            'Artist Name': artist_name,
            'Artist Followers': artist_followers,
            'Song Name': track_info['name'],
            'Album Name': track_info['album']['name'],
            'Popularity Score (0-100)': track_info['popularity'],
            'Spotify URL': track_info['external_urls']['spotify']
        })

# Remove duplicates (some songs are on multiple albums)
df_tracks = pd.DataFrame(all_tracks).drop_duplicates(subset='Song Name')

# Sort by popularity and take top 20
df_top_20 = df_tracks.sort_values(by='Popularity Score (0-100)', ascending=False).head(20)

# Export to CSV with comma delimiter
df_top_20.to_csv('spotify_artist_top_20.csv', index=False, encoding='utf-8')

print(f"✅ Export complete: spotify_artist_top_20.csv for {artist_name}")

#KEEP THIS INFORMATION IN MIND:

#| Popularity Score | Meaning                                              |
#| ---------------- | ---------------------------------------------------- |
#| **90–100**       | Extremely popular (global hits, viral songs)         |
#| **70–89**        | Very popular (frequent playlists, big artists)       |
#| **40–69**        | Moderately popular (steady listens, niche fanbase)   |
#| **0–39**         | Less popular (rarely played, older or indie content) |


✅ Export complete: spotify_artist_top_20.csv for Falling In Reverse
